In [1]:
import torch

from utils.evaluation.iou import IoU

In [2]:
# 'sofa', 'chair', 'desk', 'bed', 'bookcase', 'tool', 'misc', 'wardrobe', 'table'
category = "table"

with open("data/category.txt", "w") as file:
    file.write(category)

from data.dataloader_filtered import train_loader, test_loader, val_loader

Loading 1870 files of category table.


In [8]:
import json

with open("results/f1.json", "r") as results:
    ious = json.loads(results.read())

for cat, iou in ious.items():
    print(f"{cat}: \t{iou:.4f}")

print(sum(ious.values())/len(ious.values()))

table: 	0.3200
sofa: 	0.7437
chair: 	0.2871
desk: 	0.6563
bed: 	0.2652
bookcase: 	0.5178
tool: 	0.7217
misc: 	0.8434
wardrobe: 	0.9715
0.5918553221838321


In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)

In [ ]:
from model.model import Rec3D, version
model = Rec3D().to(device)

In [ ]:
try:
    model.load_state_dict(torch.load("model/weights/weights_{}_{}.pdf".format(version, category)))
    print("Weights loaded")
except:
    print("Weights not found")
    

Weights loaded


In [ ]:
from torchmetrics.classification import BinaryF1Score

In [ ]:
loss = BinaryF1Score().to(device)
loss_list = list()

threshold_rec = 0.2

for X, y_mask, y_cloud, image in test_loader:
    if X.shape == (1,1):
        continue

    y = model(X.to(device), torch.zeros((1,128,128,128)).to(device))

    loss_list.append(loss(y[1], y_cloud.to(device)).item())


print(f"Mean F1 score for {category} category: {sum(loss_list) / len(loss_list):.4f}")

Mean F1 score for table category: 0.2846


In [ ]:
import json

with open("results/f1.json", "r") as results:
    ious = json.loads(results.read())

if category in ious.keys():
    if sum(loss_list) / len(loss_list) > ious[category]:
        ious[category] = sum(loss_list) / len(loss_list)
else:
    ious[category] = sum(loss_list) / len(loss_list)

with open("results/f1.json", "w") as results:
    results.write(json.dumps(ious))